# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
#import the csv
output_data_file = "../output_data/cities.csv"
city_data_df = pd.read_csv(output_data_file)

,Unnamed: 0,City,Lat,Long,Country,Max Temp,Humidity,Cloud Cover %,Wind Speed,Dates
0,0,Atuona,-9.8000,-139.0333,PF,77.59,72,15,15.84,1643404950
1,1,Castro,-24.7911,-50.0119,BR,61.97,99,100,7.29,1643405004
2,2,Carnarvon,-24.8667,113.6333,AU,69.87,88,0,5.75,1643404949
3,3,Saskylakh,71.9167,114.0833,RU,0.93,89,100,6.04,1643405020
4,4,Hermanus,-34.4187,19.2345,ZA,63.81,76,0,11.45,1643404920


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Long"]].astype(float)

#create a heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = city_data_df['Humidity'],
                                dissipating=False, max_intensity=100, point_radius=1)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.

In [46]:
destinations_df = city_data_df[(city_data_df['Humidity']<85)&
                               (city_data_df['Wind Speed']<5)&
                               (city_data_df['Max Temp']<80)&
                               (city_data_df['Max Temp']>70)&
                               (city_data_df['Cloud Cover %']<10)]
destinations_df

,Unnamed: 0,City,Lat,Long,Country,Max Temp,Humidity,Cloud Cover %,Wind Speed,Dates
88,88,Kapaa,22.0752,-159.3190,US,79.12,67,0,1.01,1643405162
132,132,Lompoc,34.6391,-120.4579,US,74.70,20,0,3.44,1643405233
181,181,Salalah,17.0151,54.0924,OM,71.69,73,0,4.61,1643405308
285,285,Taree,-31.9000,152.4667,AU,75.11,83,9,0.00,1643405517
346,346,Saint-Leu,-21.1500,55.2833,RE,74.26,71,9,3.94,1643405595
364,364,Miandrivazo,-19.5167,45.4667,MG,78.40,79,9,3.31,1643405601
413,413,Narrabri,-30.3167,149.7833,AU,76.51,65,2,1.01,1643405674
459,459,Cap Malheureux,-19.9842,57.6142,MU,79.29,80,3,3.00,1643405747
461,461,Pacific Grove,36.6177,-121.9166,US,71.71,28,0,1.99,1643405808


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
#make a new dataframe with an empty column for the hotels
hotel_df = pd.DataFrame(destinations_df)
hotel_df['Hotel Name'] = ""

,Unnamed: 0,City,Lat,Long,Country,Max Temp,Humidity,Cloud Cover %,Wind Speed,Dates,Hotel Name
88,88,Kapaa,22.0752,-159.3190,US,79.12,67,0,1.01,1643405162,
132,132,Lompoc,34.6391,-120.4579,US,74.70,20,0,3.44,1643405233,
181,181,Salalah,17.0151,54.0924,OM,71.69,73,0,4.61,1643405308,
285,285,Taree,-31.9000,152.4667,AU,75.11,83,9,0.00,1643405517,
346,346,Saint-Leu,-21.1500,55.2833,RE,74.26,71,9,3.94,1643405595,
364,364,Miandrivazo,-19.5167,45.4667,MG,78.40,79,9,3.31,1643405601,
413,413,Narrabri,-30.3167,149.7833,AU,76.51,65,2,1.01,1643405674,
459,459,Cap Malheureux,-19.9842,57.6142,MU,79.29,80,3,3.00,1643405747,
461,461,Pacific Grove,36.6177,-121.9166,US,71.71,28,0,1.99,1643405808,


In [53]:
 # update params key value
params = {
"radius": 5000,
"type": 'Hotel',
"key": g_key
}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row['Lat']
    lng = row['Long']
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    hotel_data = requests.get(base_url,params=params)
    
    # convert to json
    hotel_data = hotel_data.json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_data["results"][1]["name"]
        
    except (KeyError, IndexError):
        print("No hotels here, I guess, skipping.")

In [55]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [56]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)                             
fig

Figure(layout=FigureLayout(height='420px'))